In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
#set paths
mfcc_path = 'mfccs'
train_path = os.path.join(mfcc_path, 'training')
val_path = os.path.join(mfcc_path, 'validation')
test_path = os.path.join(mfcc_path, 'test')

#set constants
n_mfcc = 20
n_dim = 32
reject_label = 'UNKNOWN'

In [3]:
all_words = list(os.listdir(train_path))

In [4]:
#20 keywords to recognize
keywords = ['up','down','left', 'right', 'stop', 'go', 'yes', 'no', 'on', 'off', 'one', 'two', 
             'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero']

#all other words will be rejection class
rejection_words = [i for i in all_words if i not in keywords]

In [5]:
#initialize arrays for the features, we will have to drop the first entries later
X_train = np.zeros(shape = (n_mfcc, n_dim))
X_val = np.zeros(shape = (n_mfcc, n_dim))
X_test = np.zeros(shape = (n_mfcc, n_dim))

y_train = np.zeros(shape = (1))
y_val = np.zeros(shape = (1))
y_test = np.zeros(shape = (1))

In [6]:
#flag for whether a word is a reject word or not
reject = False

#loop through train, validation, and test data
for i in [train_path, val_path, test_path]:
    
    #loop through all the word directories
    for j in os.listdir(train_path):
        data_path = os.path.join(i, j)
        
        #set reject flag to true if we're in the directory of a rejection word
        if (j in rejection_words):
            reject = True
            
        #load mfcc for each file in directory
        for k in os.listdir(data_path):
            file = os.path.join(data_path, k)
            mfcc = np.load(file)
            
            #append mfcc to train, validation, or test
            if (i == train_path):
                X_train = np.append(X_train, mfcc, axis = 0)
                
                #add rejection label or the word to the y arrays
                if reject:
                    y_train = np.append(y_train, reject_label)
                else:
                    y_train = np.append(y_train, j)
                    
            elif (i == test_path):
                X_test = np.append(X_test, mfcc, axis = 0)
                
                if reject:
                    y_test = np.append(y_test, reject_label)
                else:
                    y_test = np.append(y_test, j)
            elif (i == val_path):
                X_val = np.append(X_val, mfcc, axis = 0)
                
                if reject:
                    y_val = np.append(y_val, reject_label)
                else:
                    y_val = np.append(y_val, j)
                    
        #reset reject to false as we enter a new directory
        reject = False

#drop the first n_mfcc entries because they're zeros from initialization
X_train = X_train[n_mfcc:]                
X_val = X_val[n_mfcc:]
X_test = X_test[n_mfcc:]

#drop the first entry because they're also zero from initialization
y_train = y_train[1:]
y_val = y_val[1:]
y_test = y_test[1:]

#get length of arrays in terms of mfcc size
train_length = int(len(X_train) / n_mfcc)
val_length = int(len(X_val) / n_mfcc)
test_length = int(len(X_test) / n_mfcc)

#reshape arrays to be (n, 20, 32) where n is number of samples in train, validation, or test
X_train = np.reshape(X_train, newshape = (train_length, n_mfcc, n_dim))
X_val = np.reshape(X_val, newshape = (val_length, n_mfcc, n_dim))
X_test = np.reshape(X_test, newshape = (test_length, n_mfcc, n_dim))

In [20]:
print(X_train.shape)
print(y_train.shape)

print(X_val.shape)
print(y_val.shape)

print(X_test.shape)
print(y_test.shape)

(84843, 20, 32)
(84843,)
(9981, 20, 32)
(9981,)
(11005, 20, 32)
(11005,)


These shapes correspond to the number of MFCCs generated previously and are ready to be put into a neural network.

In [29]:
#save files
cwd = os.getcwd()
np.save(os.path.join(cwd, 'X_train'), X_train)
np.save(os.path.join(cwd, 'y_train'), y_train)
np.save(os.path.join(cwd, 'X_val'), X_val)
np.save(os.path.join(cwd, 'y_val'), y_val)
np.save(os.path.join(cwd, 'X_test'), X_test)
np.save(os.path.join(cwd, 'y_test'), y_test)